<a href="https://colab.research.google.com/github/hun9008/AI_TeamProject_24FW-/blob/main/Swin_trasformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.8 MB/s eta 0:00:00


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from timm import create_model

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Swin Transformer는 224x224 입력
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:13<00:00, 12388978.91it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
swin_transforemr = create_model('swin_tiny_patch4_window7_224', pretrained=True, num_classes=10)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [6]:
print(swin_transforemr)

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU(approximate='none')
            (drop1): 

In [7]:
window_attention = swin_transforemr.layers[0].blocks[0].attn

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
from torch import nn
class CustomWindowAttention(nn.Module):
    def __init__(self, original_attention):
        super().__init__()
        self.num_heads = original_attention.num_heads
        self.qkv = original_attention.qkv
        self.attn_drop = original_attention.attn_drop
        self.proj = original_attention.proj
        self.proj_drop = original_attention.proj_drop
        self.softmax = original_attention.softmax
        self.cosine_similarity = nn.CosineSimilarity(dim=-1)
    def forward(self, x, mask=None):
        B, N, C = x.shape
        # qkv 계산 및 q, k, v로 분할
        qkv = self.qkv(x).reshape(B, N, 3, C).permute(2, 0, 1, 3)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # 코사인 유사도를 계산 (q와 k는 마지막 차원에서 코사인 유사도 계산)
        attn = self.cosine_similarity(q.unsqueeze(2), k.unsqueeze(1))

        if mask is not None:
            mask = mask.unsqueeze(1)
            attn = attn.masked_fill(mask == 0, float('-inf'))

        # 소프트맥스 적용하여 attention weights 계산
        attn = self.softmax(attn)
        attn = self.attn_drop(attn)

        # attention 결과에 value(v) 적용
        x = torch.matmul(attn.unsqueeze(2), v.unsqueeze(1)).squeeze(2)

        # 최종 투사 후 드롭아웃
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [15]:
for i in range(len(swin_transforemr.layers)):
    for j in range(len(swin_transforemr.layers[i].blocks)):
        window_attention = swin_transforemr.layers[i].blocks[j].attn
        swin_transforemr.layers[i].blocks[j].attn = CustomWindowAttention(window_attention)

In [16]:
print(swin_transforemr)

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): CustomWindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
            (cosine_similarity): CosineSimilarity()
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
       

In [17]:
model = swin_transforemr.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [18]:
for epoch in range(10):
    model.train()
    for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()
    test_loss = 0.0
    correct = 0

    with torch.no_grad():
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            predicted = torch.max(outputs, 1)[1]
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            correct += (labels == predicted).sum()
    print(
        f"epoch {epoch+1} - test loss: {test_loss / len(testloader):.4f}, accuracy: {correct / len(testloader):.4f}"
    )

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.76 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.76 GiB is free. Process 25974 has 12.99 GiB memory in use. Of the allocated memory 12.11 GiB is allocated by PyTorch, and 771.34 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)